In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import sys
from pyspark.ml.regression import RandomForestRegressor
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.linalg import SparseVector, Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql import SQLContext
from pyspark.ml.feature import StringIndexer,VectorAssembler, VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.linalg import DenseVector

In [2]:
 sc = SparkContext("local[*]", "RFR")
 spark = SparkSession(sc)

In [4]:
training = spark.read.csv("rf_training.csv", inferSchema=True, header=True)
testing = spark.read.csv("rf_testing.csv", inferSchema=True, header=True)
training = training.drop('_c0', 'location', 'date').fillna(0)
dates = testing.select('date').toPandas()['date']
locations = testing.select('location').toPandas()['location']
testing = testing.drop('_c0', 'location', 'date').fillna(0)
feature_cols = training.schema.names[:-1]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
tmp = assembler.transform(training)
training = tmp["features", "new_cases"]

t_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
ttmp = t_assembler.transform(testing)
testing = ttmp["features", "new_cases"]

In [5]:
import time
start = time.time()
tree = RandomForestRegressor(labelCol="new_cases", featuresCol="features", numTrees=1000, impurity='variance')
model = tree.fit(training)
predictions = model.transform(testing)
print(time.time() - start)

24.69293212890625


In [6]:

predictions_col = predictions.select('prediction').toPandas()['prediction'].astype("int")
output = pd.concat([dates, locations, predictions_col], 1)
print(len(dates))
print(len(predictions_col))
print(output.shape)

1296
1296
(1296, 3)


In [7]:
output.to_csv("spark_predictions")

In [ ]:
sc.stop()